In [1]:
import pandas as pd
import numpy as np
import sys
import math
sys.path.append("..") 
from scipy import stats
from utils import *


In [2]:
def areaFI(countydf,county_city_rela,citydf,areatype):
    cityflow={}
    countyflow={}
    countyarea={}
    for i in countydf.iterrows():
        countyarea[(i[1]['startcode'],i[1]['endcode'])]=(i[1]['o_'+areatype],i[1]['d_'+areatype])
    for i in citydf.iterrows():
        cityflow[(i[1]['startcode'],i[1]['endcode'])]=i[1]['flow_intensity']
    for _ in range(5):
        for i in citydf.iterrows():
            origin_start_id=i[1]['startcode']
            origin_end_id=i[1]['endcode']
            start_ids=county_city_rela[origin_start_id]
            end_ids=county_city_rela[origin_end_id]
            startarea=0 #i[1]['o_'+areatype]
            startlst=[]
            endarea=0 #i[1]['o_'+areatype]
            endlst=[]
            for k in start_ids:
                for b in end_ids:
                    if((k,b) in countyarea):
                        if(k not in startlst):
                            startlst.append(k)
                            startarea+=countyarea[(k,b)][0]
                        if(b not in endlst):
                            endlst.append(b)
                            endarea+=countyarea[(k,b)][1]
            allsum=0
            for k in start_ids:
                for b in end_ids:
                    if((k,b) in countyarea):
                        p1=countyarea[(k,b)][0]/startarea
                        p2=countyarea[(k,b)][1]/endarea
                        # print(p1,p2)
                        allsum+=(p1*p2)
                        countyflow[(k,b)]=p1*p2*cityflow[(origin_start_id,origin_end_id)]
                        # allsum+=countyflow[(k,b)]
            # print(allsum)
    # print(countyflow)
    county_flow_finetune=[]
    for i in countydf.iterrows():
        county_flow_finetune.append(countyflow[(i[1]['startcode'],i[1]['endcode'])])
    countydf['county_flow_finetune']=county_flow_finetune   
    # print(sum(county_flow_finetune),sum(citydf['flow_intensity']),sum(countydf['flow_intensity']))
    return countydf

<h2>广东省

In [ ]:
region='GD'
areatype = 'area' # or 'build_area' or'pop' or 'area'

countydf=pd.read_csv('../Data/'+region+'/'+region+'_county_flow_detail.csv')
citydf=pd.read_csv('../Data/'+region+'/'+region+'_city_flow_detail.csv')
county_city_rela=load_relation('../Data/'+region+'/city_county_rela.csv')

countydf=areaFI(countydf,county_city_rela,citydf,areatype)
evaluate(countydf['county_flow_finetune'],countydf['flow_intensity'])